In [140]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [164]:
#https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/datasets/uktradecountrybycommodityexports
url = "https://www.ons.gov.uk/file?uri=%2feconomy%2fnationalaccounts%2fbalanceofpayments%2fdatasets%2fuktradecountrybycommodityexports%2fcurrent/countrybycommodityexportsoct2021.zip"

In [165]:
r = requests.get(url) 

brexit_FEB21 = zf.ZipFile(io.BytesIO(r.content))

brexit_FEB21.extractall(".\\data")

filename = brexit_FEB21.namelist()

BadZipFile: File is not a zip file

In [143]:
df = pd.read_excel(".\\data" + "\\" + filename[0], sheet_name= "Country by Commodity Export")

In [144]:
df.head()

,COMMODITY,COUNTRY,DIRECTION,2018JAN,2018FEB,2018MAR,2018APR,2018MAY,2018JUN,2018JUL,...,2020DEC,2021JAN,2021FEB,2021MAR,2021APR,2021MAY,2021JUN,2021JUL,2021AUG,2021SEP
0,0 Food & live animals,AD Andorra,EX Exports,0.00,0.00,0.01,0.00,0.00,0.00,0.01,...,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.02,0.04
1,0 Food & live animals,AE United Arab Emirates,EX Exports,17.30,15.40,19.16,16.63,16.46,14.40,17.79,...,22.44,15.03,18.20,21.61,13.58,23.01,17.09,20.73,21.27,17.72
2,0 Food & live animals,AF Afghanistan,EX Exports,0.00,0.00,0.04,0.00,0.15,0.00,0.16,...,0.44,0.13,0.25,0.03,0.02,0.08,0.13,0.00,0.00,0.00
3,0 Food & live animals,AG Antigua and Barbuda,EX Exports,0.19,0.27,0.25,0.19,0.14,0.33,0.21,...,0.30,0.23,0.28,0.28,0.20,0.17,0.17,0.23,0.19,0.29
4,0 Food & live animals,AI Anguilla,EX Exports,0.00,0.00,0.00,0.00,0.00,0.01,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [145]:
df = df.melt(id_vars = ["COMMODITY", "COUNTRY", "DIRECTION"])

df.drop("DIRECTION", axis = 1, inplace = True)

df.variable= pd.to_datetime(df.variable, format="%Y%b")

df.rename({"variable":"time", "value": "exports"},inplace = True, axis = 1)

df.COUNTRY = df.COUNTRY.str[3:]

df.exports = df.exports*1000000

In [146]:
grp = df[df.COMMODITY == 'T Total'].groupby("COUNTRY")

test = grp.agg({"exports": "sum"})

In [147]:
topcntry = test.sort_values(by = "exports", ascending = False).iloc[0:23].index.to_list()

In [148]:
df = df[df.COUNTRY.isin(topcntry)]

In [149]:
df.COMMODITY.replace({"T Total": "Total"}, inplace = True)
df.COUNTRY.replace({"United States inc Puerto Rico": "United States"}, inplace = True)
df.COUNTRY.replace({"Extra EU 28 (Rest of World)": "Excl. EU 28 (Rest of World)"}, inplace = True)

In [150]:
df

,COMMODITY,COUNTRY,time,exports
1,0 Food & live animals,United Arab Emirates,2018-01-01,1.730000e+07
12,0 Food & live animals,Australia,2018-01-01,2.238000e+07
15,0 Food & live animals,Total EU(28),2018-01-01,8.570000e+08
19,0 Food & live animals,Belgium,2018-01-01,4.396000e+07
36,0 Food & live animals,Canada,2018-01-01,1.390000e+07
...,...,...,...,...
1333072,Total,Sweden,2021-09-01,3.410000e+08
1333073,Total,Singapore,2021-09-01,3.610000e+08
1333099,Total,Turkey,2021-09-01,3.820000e+08
1333107,Total,United States,2021-09-01,3.537000e+09


In [151]:
grp = df.groupby(["COUNTRY"])

In [152]:
def ex_metal_total(df):
        
    if not (df[df["COMMODITY"] == "9 Unspecified goods"].empty):
                
        foo = pd.DataFrame([])

        fooall = df[df["COMMODITY"] == "Total"].set_index(["time"])["exports"] 
        
        foobar = df[df["COMMODITY"] == "9 Unspecified goods"].set_index(["time"])["exports"]

        foo["exports"] = fooall - foobar
    # Grab the total and then sbutract off the unspecified goods
    
        foo["COUNTRY"] = df.COUNTRY.unique()[0]
    
        #print(df.COUNTRY.unique())
    
        foo["COMMODITY"] = "Total (ex. metals)"
        
        foo.reset_index(inplace = True)
        
        foo[df.columns]
        
        df = df.append(foo)
    
    return df

In [153]:
df = grp.apply(ex_metal_total)

In [154]:
df.drop(["COUNTRY"], axis = 1, inplace = True)

df.reset_index(inplace = True)

df.drop(["level_1"], axis = 1, inplace = True)

In [155]:
file = ".\\data"+ "\\UK-exports-1997-2017.parquet"

foo = pq.read_table(file).to_pandas()

In [156]:
df = df.append(foo) #add the historical data

In [157]:
df

,COUNTRY,COMMODITY,time,exports
0,Australia,0 Food & live animals,2018-01-01,2.238000e+07
1,Australia,00 Live animals,2018-01-01,2.220000e+06
2,Australia,01 Meat & meat preparations,2018-01-01,1.800000e+05
3,Australia,02 Dairy products & eggs,2018-01-01,1.090000e+06
4,Australia,03 Fish & shellfish,2018-01-01,1.900000e+05
...,...,...,...,...
730291,Whole world,Total (ex. metals),2017-08-01,2.597800e+10
730292,Whole world,Total (ex. metals),2017-09-01,2.849800e+10
730293,Whole world,Total (ex. metals),2017-10-01,2.978700e+10
730294,Whole world,Total (ex. metals),2017-11-01,3.006700e+10


In [158]:
df.set_index(["COUNTRY", "COMMODITY", "time"], inplace = True)

In [159]:
df.sort_index(inplace = True)

df.tail()

exports
COUNTRY     COMMODITY          time                    
Whole world Total (ex. metals) 2021-05-01  2.692800e+10
                               2021-06-01  2.742600e+10
                               2021-07-01  2.677200e+10
                               2021-08-01  2.366300e+10
                               2021-09-01  2.751100e+10

In [160]:
df.head()

exports
COUNTRY   COMMODITY             time                 
Australia 0 Food & live animals 1997-01-01  4950000.0
                                1997-02-01  4870000.0
                                1997-03-01  3800000.0
                                1997-04-01  4640000.0
                                1997-05-01  4230000.0

In [161]:
out_file = ".\\data"+ "\\UK-exports.parquet"

pq.write_table(pa.Table.from_pandas(df), out_file)

In [162]:
file = ".\\data"+ "\\UK-exports.parquet"

foo = pq.read_table(file).to_pandas()

In [163]:
foo.head()

exports
COUNTRY   COMMODITY             time                 
Australia 0 Food & live animals 1997-01-01  4950000.0
                                1997-02-01  4870000.0
                                1997-03-01  3800000.0
                                1997-04-01  4640000.0
                                1997-05-01  4230000.0